In [ ]:
import websocket
import json
from urllib.parse import quote

In [ ]:
ws = websocket.WebSocket()
website_id='google.com'
ws.connect(f'wss://plankton-app-5rfza.ondigitalocean.app/websites?scraping={website_id}')
ws.recv()

In [ ]:
ws.close()

In [ ]:
ws = websocket.WebSocket()
page_id = 'https://betway.com/es/sports/evt/10647401'
website_id='betway.com'
# uri = 'wss://plankton-app-5rfza.ondigitalocean.app'
uri='ws://localhost:8000'
ws.connect(f'{uri}/games/{website_id}?game_id={page_id}')
ws.recv()

In [ ]:
from core.models import r
from redis.commands.search.query import Query, NumericFilter

In [ ]:
r.ft('idxGames').search(Query('@scraping:{true}').paging(0, 5))

In [10]:
from core.webdriver import init_driver
from selenium.webdriver.common.by import By
from core.models import Bet
driver = init_driver()
url='https://betway.com/es/sports/evt/10682257'
driver.get(url)

In [20]:
def get_bids(page_url):
    for elm in driver.find_elements(By.CSS_SELECTOR, '.collapsablePanel'):
        collapsed_elm = elm.find_element(By.CSS_SELECTOR, '.collapsableContent')
        if 'empty' in collapsed_elm.get_attribute('class'):
            try:
                elm.click()
            except:
                pass
        try:
            header_elm = elm.find_element(By.CSS_SELECTOR, '.titleText>.title')
            cashout_elm = elm.find_element(By.CSS_SELECTOR, '.cashOutMarketIndicatorContainer')
        except:
            continue    
        try:
            labels_elms = elm.find_elements(By.CSS_SELECTOR, '.outcomeHeader, .outcomeItemHeader')
            odds_elms = elm.find_elements(By.CSS_SELECTOR, '.oddsDisplay')
        except:
            continue
        
        for labels_elm, odds_elm in zip(labels_elms, odds_elms):
            try:
                group = header_elm.text.strip()
                name = labels_elm.text.strip()
                bet = Bet(
                    id=f'{group}-{name}'.replace(' ', '-').lower(),
                    websiteId='betway.com',
                    gameId=page_url,
                    group=group, 
                    name=name, 
                    bet=float(odds_elm.text.replace(',', '.'))
                )
            except Exception as e:
                print('Error', e)
            else:
                print(bet.id)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2879862164.py, line 25)

In [19]:
get_bids(url)

usc-paloma
empate
harksheide
sí
no
usc-paloma-gana-y-ambos-equipos-marcan
empate-y-ambos-equipos-marcan
Error could not convert string to float: ''
usc-paloma-gana-y-sí
empate-y-sí
harksheide-gana-y-sí
usc-paloma-gana-y-no
empate-y-no
harksheide-gana-y-no
usc-paloma-o-empate
usc-paloma-o-harksheide
empate-o-harksheide
usc-paloma-o-empate
usc-paloma-o-harksheide
empate-o-harksheide
usc-paloma-o-empate-y-sí
usc-paloma-o-harksheide-y-sí
empate-o-harksheide-y-sí
usc-paloma-o-empate-y-no
usc-paloma-o-harksheide-y-no
empate-o-harksheide-y-no
1
x
2
1-0
0-0
0-1
2-0
1-1
0-2
2-1
2-2
1-2
3-0
3-3
0-3
3-1
Error could not convert string to float: '-'
1-3
3-2
Error could not convert string to float: '-'
2-3
4-0

0-4
4-1

1-4
Error could not convert string to float: '-'

Error could not convert string to float: '-'
4-3
Error could not convert string to float: '-'
3-4
Error could not convert string to float: '-'
Error could not convert string to float: ''
Error could not convert string to float: ''
5-1